In [1]:
import math
import itertools
from collections import Counter

In [2]:
def load_alphabet():
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    alphabet_slots = [list(alphabet), list(alphabet), list(alphabet), list(alphabet), list(alphabet)]
    return alphabet_slots

In [3]:
def load_wordlist(filename, word_length):
    wordlist = []
    with open(filename) as file:
        while (line := file.readline().rstrip()):
            if len(line) == word_length:
                wordlist.append(line.upper())
#     print(f"Loaded {len(wordlist)} {word_length}-letter words to the wordlist")
    return wordlist

In [4]:
def analyze_guess(guess_input, letter_slot_list=load_alphabet(), wordlist=load_wordlist("words.txt", 5)):
    word_input = guess_input["word"].upper()
    wordle_output = guess_input["result"]
    
    yellow_letters = []
    black_letters = []
    i_prev = -1
    
    [print(letter, end="") for letter in word_input]
    print("")
    [print(results_options[color], end="") for color in wordle_output]
    print("")
    
    # iterate through the input/result pairing to classify each letter
    for idx, (letter, result) in enumerate(zip(list(word_input), wordle_output)):
#         print(results_options[result], letter) # display letter and tile color
    
        if result == 1 or result in ("g", "green"):
            letter_slot_list[idx] = [letter] # lock in greens

        elif result == 2 or result in ("y", "yellow"):
            yellow_letters.append(letter) # list of yellow letters to double check
            if letter in letter_slot_list[idx]:
                letter_slot_list[idx].remove(letter) # remove yellow from current slot

        elif result == 0 or result in ("b", "black"):
            black_letters.append(letter) # list of black letters to double check
            if letter in letter_slot_list[idx]:
                letter_slot_list[idx].remove(letter) # remove black from current slot

        else: print("[!] Entry mistake?")

    # remove black letters for which there are not also yellow letters
    for idx, letter_slot in enumerate(letter_slot_list):
        letter_slot_list[idx] = list(sorted(set(letter_slot) - set(list(set(black_letters) - set(yellow_letters)))))
    
    # create all combinations of remaining letters into possible words
    possible_combinations = ["".join(item) for item in list(itertools.product(*letter_slot_list))]
    
    # only keep the words which exist in the wordlist
    potential_solutions = list(sorted(set(possible_combinations).intersection(wordlist)))
    
    # remove words which do not include all yellow letters
    potential_solutions = [item for item in potential_solutions if all(x in item for x in yellow_letters)]
    
    print(f"\nTotal Combinations: {len(possible_combinations)}")
    print(f"Potential Solutions: {len(potential_solutions)} ({100/len(potential_solutions):.2f}% random guess)")
    
    # only display potential solutions if there are less than 200
    if len(potential_solutions) <= 200:
        # display possible solutions
        print(f"\n{potential_solutions}")
    
    # determine the number of digits of the most common occurence's count
    most_common = max([count for value, count in Counter([answer[i] for answer in potential_solutions]).most_common(1)] for i in range(0,5))
    digits = int(math.log10(most_common[0]))+1
    
    # display how many of each unused letter are left in each slot
    for i in range (0,5):
        for value, count in Counter([answer[i] for answer in potential_solutions]).most_common(15):
            single_output = f"{value}:{str(count).rjust(digits)} | "
            if i != i_prev:
                print(f"\n[{i+1}] {single_output}", end="")
            else:
                print(single_output, end="")
            i_prev = i
    print("")
    print(f"{'_'*25}\n")
    
    return potential_solutions, letter_slot_list

In [5]:
results_options = {
    0: "⚫", "b": "⚫", "black": "⚫", 
    1: "🟢", "g": "🟢", "green": "🟢", 
    2: "🟡", "y": "🟡", "yellow": "🟡"
}

In [8]:
guesses = [
    {
        "word": "SPINE",
        "result": [0,0,0,0,2]
    },
    {
        "word": "DREAM",
        "result": [0,2,2,0,0]
    },
    {
        "word": "OTHER",
        "result": [0,2,2,2,2]
    },
    {
        "word": "",
        "result": [1,0,1,0,1]
    },
    {
        "word": "",
        "result": [1,1,1,0,1]
    },
    {
        "word": "",
        "result": [0,1,1,2,0]
    }
]

for idx, guess in enumerate(guesses):
    if guess["word"] and guess["result"]:
        if idx == 0:
            solutions, letters = analyze_guess(guess, load_alphabet())
        else:
            solutions, letters = analyze_guess(guess, letters, solutions)

SPINE
⚫⚫⚫⚫🟡

Total Combinations: 4919376
Potential Solutions: 1180 (0.08% random guess)

[1] R:110 | B:103 | C:102 | D: 91 | T: 82 | F: 81 | L: 81 | H: 69 | E: 68 | A: 62 | M: 62 | G: 55 | W: 49 | O: 39 | J: 28 | 
[2] E:402 | A:212 | O:183 | R: 80 | U: 65 | L: 61 | H: 33 | M: 21 | W: 19 | Y: 18 | D: 14 | C: 13 | B: 12 | G: 12 | X: 12 | 
[3] E:191 | R:107 | L: 97 | T: 93 | D: 61 | B: 59 | M: 58 | K: 55 | V: 55 | A: 55 | W: 55 | G: 52 | C: 45 | Y: 39 | U: 37 | 
[4] E:711 | A: 97 | R: 56 | C: 46 | L: 44 | T: 39 | U: 32 | O: 32 | M: 22 | D: 18 | G: 18 | K: 14 | F: 13 | B: 11 | H:  6 | 
[5] D:305 | R:245 | Y:160 | T: 98 | L: 95 | A: 67 | H: 45 | O: 37 | M: 31 | K: 26 | X: 22 | B: 16 | W:  9 | G:  9 | C:  7 | 
_________________________

DREAM
⚫🟡🟡⚫⚫

Total Combinations: 2105352
Potential Solutions: 147 (0.68% random guess)

['BERET', 'BERKO', 'BEROB', 'BERRY', 'BERTH', 'BERYL', 'BEVER', 'BEVOR', 'BLUER', 'BOREL', 'BORER', 'BOWER', 'BOXER', 'BURET', 'BUYER', 'CEORL', 'CEROC', 'CERTY', 'COOER',